In [1]:
# prompt: driveをマウント

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [3]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

fatal: destination path 'lecture-ai-engineering' already exists and is not an empty directory.


In [4]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import random
random.seed(0)

In [7]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [18]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [12]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "SIGGRAPHとは何ですか？"
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpe

In [13]:
print(response)

SIGGRAPHは、コンピュータグラフィックス、コンピュータアート、インタラクティブシステム、ゲーム、仮想現実、人工知能など、関連する分野の研究、開発、展示を目的としている国際会議です。SIGGRAPHは、ACM（Association for Computing Machinery）の下部組織であるSIGGRAPH（Special Interest Group on Computer Graphics and Interactive Techniques）によって主催され、年に一度開催されます。

SIGGRAPHでは、世界中の研究者、開発者、学生などが、最新の技術や研究成果を発表し、展示することができます。また、会議では、展示会、ワークショップ、セミナー、パネルディスカッションなど、多くのイベントが開催されます。

SIGGRAPHは、コンピュータグラフィックスや関連する分野の研究、開発、展示の最重要のイベントとして知られており、世界中の研究者、開発者、学生などが参加することで、最新の技術や研究成果を学ぶことができます。


In [9]:
!pip install -U openai

In [10]:
gold_answers = [
    "コンピュータグラフィックス（CG）とインタラクティブ技術に関する ACM SIGGRAPH 主催の世界最大級の国際会議",
    "ハリケーンや竜巻などの大規模な大気現象。",
    "剛体接触における非平滑（クーロン）摩擦をロバストかつ高精度に数値的に解くこと。",
    "Real-Time Live! 「Best in Show」賞。",
    "タイトル: \"Repulsive Shells\" 分野: ソルバー／手法（衝突・接触処理アルゴリズム）。",
    "Neural network representation for fluid simulation　目的: ニューラルフィールドで流体場を表現し、境界条件を陽的に満たしつつ Monte Carlo 圧力ソルバーで高速・高精度に流体計算を行う。Taming Diffusion Probabilistic Models for Character Control（または “Flexible Motion In-betweening with Diffusion Models”）目的: 拡散モデルを用いてキャラクターのモーション（補間や制御）をデータ駆動で生成し、従来の物理シムでは難しい滑らかな動きを効率的に得る。"
]

In [11]:
# @title 評価実装

from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query):

        messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

# 完全性評価
def evaluate_answer_completeness(query, response, reference):
    """
    4 : User Answer が Reference の情報を漏れなく含み、等価
    2 : 主要ポイントはカバーしているが一部欠落している
    0 : ほとんど／まったくカバーしていない
    """
    prompt = (
        "Instruction: You are an expert grader for COMPLETENESS.\n"
        "Rate how completely the User Answer covers the information found in the Reference Answer "
        "for the given Question.\n"
        "Say 4 (fully complete) if *every* fact, number, and nuance in the Reference appears in the User Answer.\n"
        "Say 2 (partially complete) if the User Answer covers most key ideas but omits some details.\n"
        "Say 0 (incomplete) if it misses many important points or answers a different aspect.\n"
        "Do NOT explain—return only 4, 2, or 0.\n\n"
        "### Question: {query}\n"
        "### Reference Answer: {reference}\n"
        "### User Answer: {response}\n"
        "Score:"
    ).format(query=query, reference=reference, response=response)

    try:
        score = int(openai_generator(prompt))
    except Exception:
        score = 0
    return score

#関連性評価
def evaluate_answer_relevance(query, response):
    """
    4 : 回答が質問に直接・適切に答えていて余計な脱線がない
    2 : 部分的には答えているが、やや的外れ／不要情報が多い
    0 : ほぼ（または完全に）質問に答えていない／別トピック
    """
    prompt = (
        "Instruction: You are an expert grader for RELEVANCE.\n"
        "Given a Question and a User Answer, rate how relevant the answer is to that question.\n"
        "Say 4 (highly relevant) if the answer directly and fully addresses the question with minimal off-topic content.\n"
        "Say 2 (partially relevant) if it answers only part of the question or includes significant digressions.\n"
        "Say 0 (irrelevant) if it fails to answer the question or is largely off-topic.\n"
        "Do NOT explain—return only 4, 2, or 0.\n\n"
        "### Question: {query}\n"
        "### User Answer: {response}\n"
        "Score:"
    ).format(query=query, response=response)

    try:
        score = int(openai_generator(prompt))
    except Exception:
        score = 0
    return score

In [69]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answers[0])
print(score)

2.0


文ごとにsplitを行う

In [14]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

txtファイル読み込み

In [15]:
with open("/content/SIGGRAPH_2024_物理シミュレーション論文.txt", "r") as f:
  raw_writedown = f.read()

In [16]:
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[5])

ドキュメントサイズ:  185
ドキュメントの例: 
 採択されたTechnical Papersは、ACM Transactions on Graphics (TOG) のVolume 43, Issue 4 (2024年7月) として出版され、ACM Digital Libraryを通じてアクセス可能です 5。SIGGRAPH 2024のコンテンツは、OpenSurroundプログラムの一環として、ACM Digital Libraryでの公開後30日間は無料で利用可能でした 9。また、SIGGRAPHおよびSIGGRAPH Asiaの会議コンテンツは、通常、会議開始2週間前から終了1週間後までの1ヶ月間、ACM Digital Libraryで無料アクセスが提供されます 10。会議の論文集や関連コンテンツへのアクセスは、モバイルアプリやウェブサイトからも案内されています 9。


In [17]:
# Retrievalの実行
questions = [
    "Siggraphとは",
    "「Cyclogenesis」でシミュレーション対象となる気象現象は何ですか。",
    "「Primal-Dual Non-Smooth Friction for Rigid Body Animation」が解決を試みている物理的課題は何ですか。",
    "「Mesh Mortal Combat: Real-Time Voxelized Soft-Body Destruction」は SIGGRAPH 2024 でどの賞を受賞しましたか。",
    "SIGGRAPH 2024 で Best Paper Award を受賞した研究タイトルは何ですか。また、それはどの分野（流体 / 変形体 / ソルバーなど）に分類されますか。",
    "機械学習を利用して物理シミュレーションを加速・補強する例を、本文から 2 つ挙げて要点を比較してください。"
  ]


query_embeddings = []
document_embeddings = []
scores = []

for i,str in enumerate(questions):
  query_embeddings.append(emb_model.encode([questions[i]], prompt_name="query"))
  document_embeddings.append(emb_model.encode(documents))
  print(questions[i])

  # 各ドキュメントの類似度スコア
  scores.append((query_embeddings[i] @ document_embeddings[i].T) * 100)
  print(f"question{i+1}:{scores[i].tolist()}")

Siggraphとは
question1:[[74.78534698486328, 60.26651382446289, 77.2302474975586, 85.23372650146484, 80.16271209716797, 76.1600112915039, 69.59178161621094, 69.04000091552734, 67.06593322753906, 62.74777603149414, 73.761474609375, 73.68134307861328, 66.54332733154297, 74.7239761352539, 65.7957763671875, 65.66741180419922, 67.91773223876953, 63.129459381103516, 68.39759063720703, 63.84170150756836, 66.51339721679688, 63.913631439208984, 68.15658569335938, 65.19853973388672, 65.44267272949219, 64.85801696777344, 64.80128479003906, 63.759403228759766, 66.40144348144531, 75.98139953613281, 63.48944854736328, 65.15960693359375, 66.73410034179688, 61.17079162597656, 60.06140899658203, 72.1358871459961, 64.26210021972656, 69.74626922607422, 64.54149627685547, 66.37226104736328, 65.07742309570312, 65.55824279785156, 63.28417205810547, 64.65840148925781, 64.82926940917969, 65.24957275390625, 73.82170104980469, 63.40343475341797, 63.27351760864258, 63.121498107910156, 64.39204406738281, 63.91585922

In [19]:
topk = 5
for j in range(len(questions)):
  print(f"question{j+1}: {questions[j]}")
  for i, index in enumerate(scores[j].argsort()[0][::-1][:topk]):
    print(f"取得したドキュメント{i+1}: (Score: {scores[j][0][index]})")
    print(documents[index], "\n")
  print("\n\n")

question1: Siggraphとは
取得したドキュメント1: (Score: 85.23372650146484)
SIGGRAPH (Special Interest Group on Computer GRAPHics and Interactive Techniques) は、コンピュータグラフィックス（CG）とインタラクティブ技術に関する世界最高峰の年次会議であり、ACM SIGGRAPHによって開催されています 1。研究者、アーティスト、開発者、映画製作者、科学者、ビジネス専門家など、世界中の数千人のメンバーが集い、デジタルアートとインタラクティブデザインの未来を構築する場となっています 3。2024年の会議は、7月28日から8月1日までコロラド州デンバーで開催されました 1。 

取得したドキュメント2: (Score: 81.16515350341797)
SIGGRAPHは、これらの最先端の研究成果が発表され、議論される最高峰の場であり 2、その多くはACM Transactions on Graphics (TOG) を通じて学術コミュニティに共有されます 5。SIGGRAPH 2024で示された多岐にわたる進歩は、物理シミュレーションがコンピュータグラフィックスとインタラクティブ技術の未来において、引き続き不可欠な役割を果たしていくことを強く示唆しています。 

取得したドキュメント3: (Score: 80.16271209716797)
SIGGRAPHのプログラムは広範な分野を網羅しており、アニメーション、シミュレーション、イメージング、ジオメトリ、モデリング、レンダリング、ヒューマンコンピュータインタラクション、ハプティクス、ファブリケーション、ロボティクス、可視化、オーディオ、光学、プログラミング言語、没入型体験、視覚のための機械学習などが含まれます 5。特に、物理シミュレーションは、Technical Papersプログラムにおける重要な研究領域の一つとして位置づけられています 5。エンターテイメント、ゲーム、科学イメージング、医療応用、デザイン、写真、ファッション、建築、コミュニケーション、法医学など、多岐にわたる分野への基礎的アイデアおよび応用的貢献が歓迎されています 6。 

取得したドキュメント4: (Score: 7

In [21]:
normal_responses = []
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"

for j in range(len(questions)):
  question =  f"[質問]{questions[j]}"
  normal_responses.append(generate_output(question, system_prompt))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention

In [20]:
references = []
rag_responses = []
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"

for j in range(len(questions)):
  references.append("\n".join(["* " + documents[i] for i in scores[j].argsort()[0][::-1][:topk]]))
  question =  f"[参考資料]\n{references[j]}\n\n[質問]{questions[j]}"
  rag_responses.append(generate_output(question, system_prompt))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention

In [22]:
for i in range(len(questions)):
  print(f"question{i+1}:{questions[i]}\n")
  print(f"normal_response:\n {normal_responses[i]}\n")
  print(f"rag_response:\n {rag_responses[i]}\n")
  for j in range(50):
    print("---",end = "")
  print("\n")

question1:Siggraphとは

normal_response:
 Siggraphは、コンピュータグラフィックス（CG）や計算機科学の分野で最も権威のある国際会議の1つです。Siggraphは、Annual Conference on Computer Graphics and Interactive Techniquesの略称で、毎年夏に開催されます。

Siggraphは、1969年に始まり、以来、CGや計算機科学の分野で最も重要な会議の1つとなっています。会議では、CGの最新の技術や研究結果、計算機科学の新しい発展など、世界中の研究者や業界の専門家が参加し、発表や討論を行うことができます。

Siggraphには、多くのセッションやワークショップが開催されます。例えば、CGの新しい技術やアプリケーション、計算機科学の新しい発展や研究結果、VR/ARやゲームの最新の技術など、幅広い分野にわたるトピックが討論されます。

また、Siggraphには、賞や認定もあります。例えば、Best Paper AwardやBest Student Paper Awardなど、優秀な研究結果や学生の研究成果に対して授与される賞があります。

Siggraphは、CGや計算機科学の分野で最も権威のある会議の1つであり、世界中の研究者や業界の専門家が参加し、最新の技術や研究結果を共有する場として、非常に重要な役割を果たしています。

rag_response:
 Siggraphは、コンピュータグラフィックス（CG）とインタラクティブ技術に関する世界最高峰の年次会議です。ACM SIGGRAPHによって開催され、研究者、アーティスト、開発者、映画製作者、科学者、ビジネス専門家など、世界中の数千人のメンバーが集い、デジタルアートとインタラクティブデザインの未来を構築する場となっています。

------------------------------------------------------------------------------------------------------------------------------------------------------

question2:「Cyclogenesis」でシミュレーション対象となる気象現

In [23]:
# 評価の実行
for i in range(len(questions)):
    accuracy_score = evaluate_answer_accuracy(questions[i], normal_responses[i], gold_answers[i])
    completeness_score = evaluate_answer_completeness(questions[i], normal_responses[i], gold_answers[i])
    relevance_score = evaluate_answer_relevance(questions[i], normal_responses[i])

    print(f"Question {i+1}: {questions[i]}")
    print(f"- Normal Response Accuracy: {accuracy_score}")
    print(f"- Normal Response Completeness: {completeness_score}")
    print(f"- Normal Response Relevance: {relevance_score}")

    accuracy_score_rag = evaluate_answer_accuracy(questions[i], rag_responses[i], gold_answers[i])
    completeness_score_rag = evaluate_answer_completeness(questions[i], rag_responses[i], gold_answers[i])
    relevance_score_rag = evaluate_answer_relevance(questions[i], rag_responses[i])

    print(f"- RAG Response Accuracy: {accuracy_score_rag}")
    print(f"- RAG Response Completeness: {completeness_score_rag}")
    print(f"- RAG Response Relevance: {relevance_score_rag}")
    print("-" * 50)


Question 1: Siggraphとは
- Normal Response Accuracy: 1.0
- Normal Response Completeness: 2
- Normal Response Relevance: 4
- RAG Response Accuracy: 2.0
- RAG Response Completeness: 2
- RAG Response Relevance: 4
--------------------------------------------------
Question 2: 「Cyclogenesis」でシミュレーション対象となる気象現象は何ですか。
- Normal Response Accuracy: 1.0
- Normal Response Completeness: 0
- Normal Response Relevance: 4
- RAG Response Accuracy: 4.0
- RAG Response Completeness: 4
- RAG Response Relevance: 4
--------------------------------------------------
Question 3: 「Primal-Dual Non-Smooth Friction for Rigid Body Animation」が解決を試みている物理的課題は何ですか。
- Normal Response Accuracy: 1.0
- Normal Response Completeness: 2
- Normal Response Relevance: 4
- RAG Response Accuracy: 2.0
- RAG Response Completeness: 2
- RAG Response Relevance: 4
--------------------------------------------------
Question 4: 「Mesh Mortal Combat: Real-Time Voxelized Soft-Body Destruction」は SIGGRAPH 2024 でどの賞を受賞しましたか。
- Normal Response Accu